In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from copy import deepcopy
import random
import numba as nb
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def norm(a):
    return a / np.linalg.norm(a)

def reconstruct(mps_in, ax=True):
    state_vec = []

    mps = deepcopy(mps_in)

    N = len(mps)

    # this converts to a different index order
    # where the physical index is the outermost index and the bond index is contained
    # also puts into canonical form we are used to seeing

    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)


    for i in range(2**N):
        ind = [int(bit) for bit in format(i, '0{}b'.format(N))]
        prod = mps[-1][ind[-1]]
        for j in range(N-2, 0, -1):
            tensors = mps[j]
            prod = tensors[1] @ prod
            prod = tensors[0][ind[j]] @ prod
        prod = mps[0][1] @ prod
        prod = mps[0][0][:, ind[0]] @ prod
        state_vec.append(prod)
    return norm(state_vec)

def lcanonical(m):
    # this converts to a different index order
    # where the physical index is the outermost index and the bond index is contained
    # also puts into canonical form we are used to seeing
    mps = deepcopy(m)
    N = len(m)
    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)
    r = []
    for i in range(len(mps)-1):
        r.append([mps[i][0][0] @ mps[i][1], mps[i][0][1] @ mps[i][1]])
    r.append(mps[-1])
    return r

def rcanonical(m):
    mps = deepcopy(m)
    N = len(m)
    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)
    r = [mps[0][0]]
    for i in range(len(mps)-2):
        r.append([mps[i][1] @ mps[i+1][0][0], mps[i][1] @ mps[i+1][0][1]])
    r.append(mps[-2][1] @ mps[-1])
    return r

def form(m):
    state_vec = []

    mps = deepcopy(m)

    N = len(mps)

    for i in range(2**N):
        ind = [int(bit) for bit in format(i, '0{}b'.format(N))]
        prod = [[row[ind[-1]]] for row in mps[-1]]
        for j in range(N-2, -1, -1):
            prod = mps[j][ind[j]] @ prod

        state_vec.append(prod[0])

    return norm(state_vec)


def mps(state_v, chi=1e50):
    mps_v = []
    N = int(np.log2(len(state_v)))
    right = np.reshape(state_v, (2, 2**(N-1)))
    for i in range(N):
        if i == N-1:
            mps_v.append(right)
            continue
        # print(right.shape)
        gamma, S, right = np.linalg.svd(right, full_matrices=False)
        # left and right most gammas are our MPS caps, only have one bond index
        if i > 0 and i < N-1:
            if chi < len(S) and chi >= 1:
                gamma = gamma[:, :chi]
                S = S[:chi]
                right = right[:chi, :]

            gamma = np.reshape(gamma, (int(gamma.shape[0]/2), 2, gamma.shape[1]))

        # don't want to reshape this way when there is only 1 column
        if right.shape[1] > 2:
            right = np.reshape(right, (int(right.shape[0]*2), int(right.shape[1]/2)))
        
        lambd = np.diag(norm(S))
        mps_v.append([gamma, lambd])

    return mps_v

@nb.njit('complex128[:](int_)', parallel=True)
def haar_state(N):
    real_part = np.empty(2**N)
    imag_part = np.empty(2**N)

    # Parallel loop
    for i in nb.prange(2**N):
        real_part[i] = np.random.normal()
        imag_part[i] = np.random.normal()

    state = real_part + 1j * imag_part
    return state

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
def rydberg_hamiltonian(N):
    # start with a 2^N x 2^N matrix
    # factor into rank 4 tensors with rank 3 caps
    # each tensor acts on a bra and ket of corresponding MPS site
    

In [ ]:
def sweep(m):
    